In [1]:
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import json

In [2]:
def get_product_name(product_URL) : 
    dom = requests.get(product_URL)
    soup = BeautifulSoup(dom.text, "html.parser")
    result = soup.find("h1").text
    return result

In [3]:
def get_total_number_of_product_reviews(product_URL) : 
    dom = requests.get(product_URL)
    soup = BeautifulSoup(dom.text, "html.parser")
    result = soup.select_one("#li_review_text > a > span").text
    return int(result[1 : -1].replace(',', ''))

In [4]:
def get_product_number(product_URL) : 
    start_num = product_URL.find("i_sProductcd=")
    product_number = product_URL[start_num + len("i_sProductcd=") : ]
    return product_number

In [5]:
def scrape_product_reviews(product_URL) : 
    
    print("scraping start")
    df = pd.DataFrame(columns = ["user_id", "channel", "option", "review", "membership", "recom_point"])
    product_name = get_product_name(product_URL)
    base_URL = 'https://www.aritaum.com/rnl/us/rnl_us_prodUseReviewList_ajax.do?'
    
    product_number = get_product_number(product_URL)
    # Your header information
    headers={
                "accept": "application/json, text/javascript, */*; q=0.01",
                "accept-encoding": "gzip, deflate, br",
                "accept-language": "ko,en-US;q=0.9,en;q=0.8",
                "content-length": "153",
                "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
                "cookie": "WMONID=Da-uQHZZ8Xe; JSESSIONID=9632B74971DD443B60A536EFB06985F7; org.springframework.web.servlet.i18n.CookieLocaleResolver.LOCALE=ko; noMbrPcId=201809231354242473403; optimizelyEndUserId=oeu1537678493225r0.6548219971589961; wcs_bt=s_51fdc9a13f4b:1537678493; _ga=GA1.2.874480751.1537678494; _gid=GA1.2.2130736539.1537678494; RB_PCID=1537678493991619712; RB_SSID=t3VjV19RqL; s_cc=true; s_fid=7E85E19BFC96C086-32251EC3919909F9; s_ev8=N; s_ppn=aritaum-kr%5E%EC%87%BC%ED%95%91%5E%EC%83%81%ED%92%88%EC%83%81%EC%84%B8; _gcl_au=1.1.1649061979.1537678495; RB_GUID=d1d4f3ec-6bf1-4fb3-83eb-641862e83f11; _dc_gtm_UA-110770460-4=1; _aprtetSessId=02kNu; _aprtetSessPageSeq=0; scs=%7B%22t%22%3A1%7D; insdrSV=1; current-currency=KRW; ins-mig-done=1; ins-gaSSId=2a4dad94-b11f-be6e-20c1-4138590514a3_1537682097; spUID=15376784970770b24cee272.8d3270bb; s_sq=ap-w-aritaum-prd%2Cap-w-aptotal-prd%3D%2526pid%253Daritaum-kr%25255E%2525EC%252587%2525BC%2525ED%252595%252591%25255E%2525EC%252583%252581%2525ED%252592%252588%2525EC%252583%252581%2525EC%252584%2525B8%2526pidt%253D1%2526oid%253Dhttps%25253A%25252F%25252Fwww.aritaum.com%25252Fshop%25252Fpr%25252Fshop_pr_product_view.do%25253Fi_sProductcd%25253DP0000000000000000768%252523%2526ot%253DA",
                "origin": "https://www.aritaum.com",
                "referer": "https://www.aritaum.com/shop/pr/shop_pr_product_view.do?i_sProductcd=P0000000000000000768",
                "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36",
                "x-requested-with": "XMLHttpRequest",
            }

    total_number_reviews = get_total_number_of_product_reviews(product_URL)
    print("total_number_reviews : ", total_number_reviews)
    for i in range(1, round(total_number_reviews/10) + 2):
        StartRownum = ( i - 1) * 10 + 1
        EndRownum = StartRownum + 9
        form_data = {
            'i_iNowPageNo': str(i), 
            'i_iPageSize': '10', 
            'i_iEndRownum': str(EndRownum), 
            'i_iRecordCnt': total_number_reviews, 
            'i_iTotalPageCnt': round(total_number_reviews/10) + 1, 
            'i_iStartRownum': str(StartRownum), 
            'i_sProductcd': product_number, 
            'i_sProdTypecd': '0001',
            }
        # get response
        response = requests.post(url = base_URL, data = form_data, headers = headers)
        data = response.text
        dic = json.loads(data)

        review_list = dic['object']['useReview']['useReviewList']
        for review in review_list:
            data = {
                "user_id" : review['v_userid'], #userid
                "channel" : review['v_reg_channel'], #channel
                "option" : review['v_optionnm'], #product option
                "review" : review["v_content"], #review
                "membership" : review["v_levelnm"],
                "recom_point" : review["n_recom_point"],
                
            }

            df.loc[len(df)] = data
    
    print("scraping finished")
    return  df.to_csv(path_or_buf = "Aritaum_" + product_name + ".csv", index = False, encoding = "utf-8-sig")

    

In [ ]:
# 뽀오얀 미소 발효 립 & 아이 리무버
scrape_product_reviews("http://www.aritaum.com/shop/pr/shop_pr_product_view.do?i_sProductcd=P0000000000000000768")

scraping start
total_number_reviews :  3121


----